# A note on random forest (and decision trees)

*Author* : __Kevin (December 2019)__

We mainly use random forest for our analysis. From mathematical point of view, random forest is not well calibrated: it is usually overconfident and the posterior probability estimate resembles more of a step function that overshoots the true posterior probability. For this reason, we should prefer neural network when training data is abundant. However, random forest has been proven to work really well when the training data are limited (as in our case) or computational resources are constrained.

Here we review some key ideas and properties of random forest, decision tree, as well as give some further information about our cross-validation strategy. In particular, random forest gives high prediction accuracy without parameter tuning, fast at train and test time, and has an in-built accuracy measure and interpretation tool that we can use to explain the predictions.

## Decision Tree

a.k.a. Classification and regression tree (CART).

The basic idea is to recursively split the feature space according to some binary criteria. Such a partitioning of the feature space can be represented as a decision tree. The resulting regions due to the partitioning correspond to the leaf nodes of the tree. In other words, we apply a series of yes/no questions to
+ iteratively partition the feature space at train time (i.e. build a decision tree)
+ classify query at test time (by traversing the tree)

Note that this process assumes that the training data is representative of what we want to predict (i.e. the test data should be from the same distribution as the training set). There are some arbitrary choices involved:
+ greedy tree construction (because finding the optimal tree according to some criteria is NP-hard, i.e. impractical)
+ binary decision tree based on binary splits (yes/no questions): each node has two children
+ split into contiguous regions in feature space (i.e. only one parameter per node/split)
+ single feature for each split (monothetic decision tree) as opposed to oblique decision tree (with multiple features each split)
+ impurity criteria for evaluating splits

Several impurity criteria exist, the most popular ones are misclassification error, entropy, and Gini criteria. Gini is the most popular choice (and default in `scikit-learn`) for classification problem, while mean decrease accuracy is often used for regression problem. The Gini criteria is the expected error if we randomly pick a sample with probability $p(y)$ from the node and use its label for the node:
$$ \begin{split}
\sum_y p(y) \left(\sum_{\tilde{y} \neq y} p(\tilde{y})\right) & = \sum_y p(y)(1-p(y))\\
 & = \sum_y p(y) - \sum_y p^2(y) \\
 & = 1 - \sum_y p^2(y)
\end{split}$$

Plotting it against $p(y)$, one gets a parabola for the Gini criteria (with peak at $p(y) = 0.5$, a triangle for the misclassification accuracy, and infinite slope at origin/unity for the entropy. For further details, see [The Elements of Statistical Learning by Hastie, Tibshirani, and Friedman](https://web.stanford.edu/~hastie/ElemStatLearn/).

## Tree construction and its properties

Priority for split candidates is measured by the possible reduction in (node-normalized) impurity:

$$ H - H(L) \frac{N_L}{N_L + N_R} - H(R) \frac{R}{N_L + N_R} $$

where
- $N_L$ is the amount of samples that end up in the left node
- $N_R$ is the amount of samples that end up in the right node.
- $H$ is the impurity of the node under consideration
- $H(R)$ is the impurity of the right child
- $H(L)$ is the impurity of the left child

In the absence of normalization, the resulting tree would be very deep (too many splits), with each region containing only a few samples.

The computational complexity of tree construction for *monothetic* decision trees is as follows. At the root node (level 0), we sort $n$ samples according to each of $p$ features. Since sorting is $\mathcal{O}(n \log{n})$ and measurement of impurity is $\mathcal{O}(n)$, we have a total complexity of $\mathcal{O}(p n \log{n})$. Now, if each child receives half of the samples, then at level 1, we have for the left child and right child:
$$ \begin{split} \mathcal{O}\left(p\frac{n}{2} \log{\frac{n}{2}} + p\frac{n}{2} \log{\frac{n}{2}} \right) \\
 = \mathcal{O}\left(p n \log{\frac{n}{2}} \right)
\end{split}$$

By recursive argument, we have $\mathcal{O}\left(p n \log{\frac{n}{4}} \right)$ for level 2, and so on. Therefore, for a balanced binary tree with $\lg n$ levels (logarithm with base 2), overall training complexity is $ \mathcal{O}\left(p n (\lg{n})^2 \right) $. During prediction, the computational complexity is:
+ best case: $\lg n$, when the tree always sends half the samples to the left child and the other half to the right child. For example, 1024 samples would need a tree of depth 10.
+ worst case: $n$, when each split sends only one sample to one of the children every time.
+ average case: $\mathcal{O}(\lg n)$, i.e. the tree is usually not very imbalanced. Note that this is cheap: c.f. k-NN with cost $\mathcal{O}(n)$.


The most popular kind of splits is axis-orthogonal, where only a single feature is considered at every node/split. The advantage of an axis-orthogonal split: it is invariant w.r.t. scaling of the axes, i.e. choice of feature units can be arbitrary. However, the drawback is that several splits are required for correlated features. This can be overcome by introducing *oblique* splits, which ask if the inner product of some weight vector and the feature vector satisfies a condition:

$$ w^T x > b $$

Alternatively, one could preprocess the data with Principal Component Analysis (PCA). Nevertheless, a decision tree is likely to overfit if it is trained to purity because it would do several splits to account for each observation. Thus it has low bias but high variance. Two common regularization methods to address the high variance problem are:
1. *Early stopping*: abort if node has too few samples, or if impurity reduction is too small, etc.
2. *Pruning*: build a tree and cut it back.

Nevertheless, the state-of-the-art method to reduce variance of decision trees while maintaining the low bias is to create an ensemble of decision trees (a.k.a. **random forest**).

## Random Forest

Random Forest is proposed by [Leo Breiman, 2001](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm) and works by injecting two sources of randomness:
1. sample with replacement from original training data (i.e. bootstrapping) such that each tree gets different training data. In ML speak, this is called *bagging* (quasi-short for bootstrap aggregation).
2. consider only a random subset of features to find the "optimal" split in each tree, at each node.

Due to the bagging procedure, there are samples that a tree has not seen during training time. One can use these unseen samples to make predictions to get **out-of-bag** error, which gives an in-built estimate of generalization performance (c.f. cross-validation). Now due to random split selection, one can count, for each feature, how often it is used for splitting: this gives a measure of how important a feature is in making predictions (**Gini importance**). Note that Gini importance is biased towards features with many categorical levels, but it is still useful and cheap to compute.

Prediction can be made using majority vote. However, an artifact of this approach is that the estimates in the "hard" regions where impurity is relatively high cannot be interpreted as posterior probability, because random forest is likely to be overconfident.

Additional remarks:
- In greedy tree construction, one iteratively finds the (local) best split at each iteration.
- Bagging and random split selection are the two key factors that enable random forest.
- Two hyperparameters of random forest: number of trees and size of random subset. However usually it works fine with the default values (256 trees and $\lg n$ feature subset) and tuning these values do not have significant impact on performance.
- RF can handle both categorical and continuous features simultaneuously

## Cross-validation

Our strategy for model/pipeline selection and hyperparameter tuning is to use cross-validation. Ideally, if one is rich in training data, the data can be simply **randomly** partitioned (e.g. 60% training set, 20% validation set, and 20% test set) and iterate the following:
- adjust hyperparameters, train on training data,
- evaluate classifier on validation data,

until accuracy on validation set looks OK. The final accuracy (of the best performing classifier) is evaluated on the test data **only** once. In the case when deployment is needed, the classifier is usually retrained on training data $\cup$ validation data $\cup$ test data.

On the other hand, if one has only limited training data (which is usually the case), cross-validation is the preferred strategy. Here, we partition the data randomly into $k$ folds ($k = 10$ in our case) and iterate the following:
- adjust hyperparameters,
- loop over folds: for 1 to k do
    * train on all but one fold,
    * evaluate the accuracy on the held-out fold,
- estimate the mean and variance of our accuracy,

until satisfied. Now that we have the set of best-performing hyperparameters, retrain classifier on entire data and deploy. Note that the estimate of "variance" here does not imply that we have a Gaussian distribution, because accuracy is within 0 to 100% while Gaussian has a tail. It is also worth mentioning that various theoretical estimates of how much overfitting arises due to one's choice of hyperparameters exist, e.g. Akaike Information Criterion/Bayes Information Criterion, Network Information Criterion, etc. However, these theoretical estimates make assumptions of the data/model, which usually do not hold in practice. The ML community finds that one should avoid using these theoretical estimates when building a system that is supposed to work.

## Regression

In our settings, we are dealing with regression problem, where we want to predict a response (a.k.a. dependent variable) from known explanatory variables (a.k.a. independent variables/features/regressors, etc). Since this problem is encountered in all sorts of fields, it has accordingly different names and nomenclatures. Note that prediction can also be called interpolation/extrapolation, estimation (in statistics), etc.

The mechanisms for decision tree/random forest and cross-validation above hold true for both regression and classification problems.